# Configuraciones planetarias

Los planetas, vistos desde la Tierra, tienen un comportamiento muy característico. El momento en el que los planetas están cerca del Sol se denomina **conjunción** (también aplica a encuentros aparentes de otros objectos).

Cuando un planeta se encuentra en la posición diametralmente opuesta al Sol se dedomina **oposición**.

La separación angular aparente en el cielo entre el Sol y un planeta se denomian **elongación**. En una conjunción, la enlongación es un ángulo cercano a .

Desde la antiguedad se sabe que hay dos tipos de planetas, basándose en sus configuraciones planetarias.

Hay planetas, denominados **inferiores**, que:
 * tienen dos conjunciones (pasan por delante y por detrás del Sol)
 * su elongación tiene un valor máximo (nunca se separan demasiado del Sol)
 * nunca se encuentran en oposición

Estos planetas son **Mercurio** y **Venus**.


Los planetas **superiores**:
 * tienen una conjunción (pasan por detrás del Sol)
 * su elongación va de 0$^\circ$ a 180$^\circ$ (aproximadamente)
 * tienen un oposición
 
Estos planetas son **Marte**, **Júpiter**, **Saturno**, **Urano** y **Neptuno**.

Desde el desarrollo del modelo heliocéntrico, sabemos que el movimiento aparente de los planetas se debe a la combinación de los movimientos reales de la Tierra y del planeta. 

Además, los planetas inferiores son aquellos cuya órbita es interna a la de la Tierra y los superiores, aquellos cuya órbita es externa.






[![text](https://upload.wikimedia.org/wikipedia/commons/f/f6/Positional_astronomy.svg)](https://upload.wikimedia.org/wikipedia/commons/f/f6/Positional_astronomy.svg)


<a href="https://commons.wikimedia.org/wiki/File:Positional_astronomy.svg" title="via Wikimedia Commons">Wmheric</a> / <a href="https://creativecommons.org/licenses/by-sa/3.0">CC BY-SA</a>


In [ ]:
from IPython.display import HTML, Image, display

In [ ]:
# Código Python Permite mostrar HTML en las celdas
display(Image(url='https://upload.wikimedia.org/wikipedia/commons/f/f6/Positional_astronomy.svg'))
display(HTML("""<a href="https://commons.wikimedia.org/wiki/File:Positional_astronomy.svg" title="via Wikimedia Commons">Wmheric</a> / <a href="https://creativecommons.org/licenses/by-sa/3.0">CC BY-SA</a>"""))


## Periodo sinódico

El periodo en el que se repiten las configuraciones planetarias se denomina **periodo sinódico**. Este periodo es la combinación de los periodos orbitales de la Tierra ($T_1$) y el planeta en particular ($T_2$). 


$$
\frac{1}{P} = \left| \frac{1}{T_1} - \frac{1}{T_2} \right|
$$


In [ ]:
import numpy as np
import math
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
plt.style.use('seaborn-pastel')

In [ ]:
# Semiejes mayores y periodos de los planetas
# en au y años
planets = {
    "Mercury": (0.387098, 0.240846),
    "Venus": (0.723332, 0.615198),
    "Earth": (1.000001, 1.000017),
    "Mars": (1.523679, 1.88082),
    "Jupiter": (5.2044, 11.862),
    "Saturn": (9.5826, 29.4571),
    "Uranus": (19.2184, 84.0205),
    "Neptune": (30.11, 164.8),
}

In [ ]:
def calc_e(a1, a2, ps_a):
    """Compute elongation"""
    aa = math.sqrt(a1**2 + a2**2 - 2 *a1*a2*math.cos(ps_a))
    sine = a2 / aa * math.sin(ps_a)
    return math.asin(sine)


def calc_ang(t, a1, a2, syn):
    """Compute three angles"""
    ps = math.fmod(t, syn) / syn
    u = 2 * math.pi * ps
    if a2 >= a1:
        e = calc_e(a2, a1, u)
    else:
        e = calc_e(a1, a2, u)
    if u > math.pi:
        u = 2 * math.pi - u
    e2 = np.rad2deg(abs(e))
    u2 = np.rad2deg(u)
    e3 = 180 - (e2 + u2)
    return t, u2, e2, e3

In [ ]:
def syn_period(t1, t2):
    """Synodic period"""
    syn = 365.25 * 1 / abs(1 / t1 - 1 / t2)
    return syn

In [ ]:
def plot_syn(planets, origin='Earth'):

    import math
    
    def round_to_1(x):
        n10 = 10**math.floor(math.log10(x))
        return math.ceil(x / n10) * n10
        
    tmin = 0.1
    tmax = 200
    ao, to = planets[origin]
    ta = np.logspace(np.log10(tmin), np.log10(to * 0.9))
    tb = np.logspace(np.log10(to * 1.1), np.log10(tmax))
    syna = syn_period(to, ta)
    synb = syn_period(to, tb)
    tes = []
    for name, (a,t) in planets.items():
        if name == origin:
            continue
        tes.append(t)
    pes = syn_period(to, np.asarray(tes))
    p_max = max(pes)
    
    fig = plt.figure()
    ax = plt.axes(xlim=(tmin, tmax), ylim=(0, round_to_1(p_max)))
    ax.set_xscale("log")
    ax.plot(ta, syna)
    ax.set_prop_cycle(None)
    ax.plot(tb, synb)
    ax.set_xlabel("Orbital period (years)")
    ax.set_ylabel("Synodic period (days)")
    ax.scatter(tes, pes, c='r')
    ax.axvline(to, c='green', ls='--')
    ax.set_title("Synodic periods from {}".format(origin))
    
    
    plt.show()

In [ ]:
plot_syn(planets, origin='Earth')

In [ ]:
def plot_elong(a1, p1, n1, a2, p2, n2):
    """Plot elongation and phase of planet2 from planet 1
    
    Plot elongation and phase with semimajor axis a2, period p2 and
    name n2 from planet 1 with semimajor axis a1, period p1 and
    name n1
    
    """

    syn = syn_period(p1, p2)

    p1_d = p1 * 365.25
    p2_d = p2 * 365.25
    o1_d = 2 * math.pi / p1_d
    o2_d = 2 * math.pi / p2_d

    tt = np.arange(0, syn + 1)

    tots = [calc_ang(t, a1, a2, syn) for t in tt]
    ta, a, b, c = zip(*tots)

    clors = plt.rcParams['axes.prop_cycle'].by_key()['color']
    if a2 > a1:
        p_ang, e_ang = b, c
    else:
        e_ang, p_ang = b, c

    fig, ax1 = plt.subplots()
    ax1.set_title('{}, elongation'.format(n2))
    ax1.plot(tt, e_ang, 'r')
    ax1.set_ylabel('elongation')
    plt.show()

    fig, ax1 = plt.subplots()
    ax1.set_title('{}, phase'.format(n2))
    pha = 0.5 * (1 + np.cos(np.deg2rad(p_ang)))
    p1ax = ax1.plot(tt, pha, clors[0], label='Fraction')
    ax1.set_xlabel('synodic period (days)')
    ax1.set_ylabel('phase (fraction)')
    ax2 = ax1.twinx()
    p2ax = ax2.plot(tt, p_ang, clors[1], label='Angle')
    ax2.set_ylabel('phase (angle)')
    lns = p1ax + p2ax
    labs = [l.get_label() for l in lns]
    ax1.legend(lns, labs)
    ax1.legend(lns, labs)
    plt.show()

    
def plot_elong_name(name1, name2):
    """Plot elongation and phase of planet name2 from planet name1"""
    a1, p1 = planets[name1]
    a2, p2 = planets[name2]
    if name1 != name2:
        plot_elong(a1, p1, name1, a2, p2, name2)
    else:
        raise ValueError("We can't observe {} from {}".format(name1, name2))

In [ ]:
plot_elong_name('Earth', 'Venus')


In [ ]:
def plot_anim(a1, p1, n1, a2, p2, n2):

    fig = plt.figure()
    ul = 1.1 * math.ceil(10 * max(a1, a2)) / 10
    ax = plt.axes(xlim=(-ul, ul), ylim=(-ul, ul))
    ax.set_aspect('equal')
    
    # Rad in axis coordinates
    rax = 0.012
    m1 = ax.transAxes.transform([(0.5, 0), (0.5, rax)])
    m2 = ax.transData.inverted().transform(m1)
    # Rad in data coordinates
    rdata = abs(m2[1,1] - m2[0,1])

    pl1 = plt.Circle((0, 1), rdata, fc='b')
    pl2 = plt.Circle((0, 2), rdata, fc='g')
    sun = plt.Circle((0, 0), 1.2*rdata, fc='k')

    syn = syn_period(p1, p2)

    p1_d = p1 * 365.25
    p2_d = p2 * 365.25
    o1_d = 2 * math.pi / p1_d
    o2_d = 2 * math.pi / p2_d

    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
    phase_text = ax.text(0.02, 0.90, '', transform=ax.transAxes)
    ax.text(0.02, 0.85, 'Ps={:.0f} d'.format(syn), transform=ax.transAxes)
    #energy_text = ax.text(0.02, 0.90, '', transform=ax.transAxes)

    def init():
        ax.add_patch(sun)
        o1 = plt.Circle((0,0), a1, fill=False, lw=1)
        o2 = plt.Circle((0,0), a2, fill=False, lw=1)
        ax.add_patch(o1)
        ax.add_patch(o2)

        pl1.center = (0, a1)
        pl2.center = (0, a2)
        ax.add_patch(pl1)
        ax.add_patch(pl2)
        time_text.set_text('')
        phase_text.set_text('0.0')
        return pl1, pl2, time_text, phase_text

    def animate(i):
        steps = 400.0
        frac = i / steps * (2*syn)
        x, y = pl1.center
        phase1 = o1_d * frac
        x = a1 * np.cos(phase1)
        y = a1 * np.sin(phase1)
        pl1.center = (x, y)
        x, y = pl2.center
        phase2 = o2_d * frac
        x = 0 + a2 * np.cos(phase2)
        y = 0 + a2 * np.sin(phase2)
        pl2.center = (x, y)
        
        frac, u2, e2, e3 = calc_ang(frac, a1, a2, syn)
        if a2 > a1:
            p_ang, e_ang = e2, e3
        else:
            e_ang, p_ang = e2, e3
        ps = math.fmod(frac, syn) / syn
        time_text.set_text('t = {:.0f} d fs = {:3.2f}'.format(frac * 1.0, ps))
        phase_text.set_text("E={:.2f}".format(e_ang))
        return pl1, pl2, time_text, phase_text

    anim = FuncAnimation(fig, animate, init_func=init, repeat=False,
                               frames=400, interval=20, blit=True)
    plt.close(anim._fig)
    return anim


def plot_anim_name(name1, name2):
    
    a1, p1 = planets[name1]
    a2, p2 = planets[name2]
    return plot_anim(a1, p1, name1, a2, p2, name2)


In [ ]:
anim = plot_anim_name('Earth', 'Mars')

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
# Intentamos crear una animación de plot_sy
interact(plot_syn)

In [ ]:
# Fijamos el primer argumento
interact(plot_syn, 
         planets=widgets.fixed(planets)
        );

In [ ]:
# Podemos pasar una lista de valores
interact(plot_syn, 
         planets=widgets.fixed(planets), 
         origin=['Earth', 'Mars']);

In [ ]:
# Como disponemos del diccionario de planetas
# extraemos los nombres de ahí
interact(plot_syn, 
         planets=widgets.fixed(planets), 
         origin=planets.keys()
        );

In [ ]:
w1 = widgets.Dropdown(options=list(planets.keys()), 
                      value='Earth', 
                      description='Planet1:',
                      disabled=False)
w2 = widgets.Dropdown(options=list(planets.keys()), 
                      value='Mercury', 
                      description='Planet2:',
                      disabled=False)

In [ ]:
widgets.interact(plot_elong_name, name1=w1, name2=w2);

In [ ]:
ww1 = widgets.Dropdown(options=list(planets.keys()), 
                       value='Earth', 
                       description='Planet1:',
                       disabled=False)
ww2 = widgets.Dropdown(options=list(planets.keys()), 
                       value='Mercury', 
                       description='Planet2:',
                       disabled=False)

def remove_planet1(event):
    op = list(planets.keys())
    op.remove(ww1.value)
    ww2.options = op

remove_planet1(None)

ww1.observe(remove_planet1, names='value')

In [ ]:
widgets.interact(plot_elong_name, name1=ww1, name2=ww2);